In [174]:
''' 
Constructing SEBA
'''

%reload_ext autoreload
%autoreload 2

In [175]:
import sys
sys.path.insert(1,'./src/') 

import unittest
import numpy as np
import methods
import glob
import matplotlib.pyplot as plt
import scipy
import os

In [176]:
# inputs.
V_orig = np.random.random((100,3))

# start...
V = V_orig
Rinit = None

def soft_threshold(z, mu):
        assert len(np.shape(z)) <= 1 # only accept scalars or vectors

        temp = np.zeros(np.shape(z))
        if len(np.shape(z)) == 1:
                for i in range(len(z)):
                        temp[i] = np.sign(z[i]) * np.max([np.abs(z[i]) - mu, 0])
        else:
                temp = np.sign(z) * np.max([np.abs(z) - mu, 0])        
        
        return temp

# Begin SEBA algorithm
maxiter = 5000   # maximum number of iterations allowed
F,_ = np.linalg.qr(V) # Enforce orthonormality
V = F # (!) needed?
(p,r) = np.shape(V)
mu = 0.99 / np.sqrt(p)

S = np.zeros(np.shape(V))

# Perturb near-constant vectors
for j in range(r):
        if np.max(V[:, j]) - np.min(V[:, j]) < 1e-14:
                V[:, j] = V[:, j] + (np.random.random((p, 1)) - 1 / 2) * 1e-12

# is R correct?

# ...
# Initialise rotation
if Rinit == None:
        Rnew = np.eye(r) # depends on context?
else:
        # Ensure orthonormality of Rinit
        U, _, Vt = np.linalg.svd(Rinit)
        Rnew = np.matmul(U , Vt)

#preallocate matrices
R = np.zeros((r, r))
Z = np.zeros((p, r))
Si = np.zeros((p, 1))

iter = 0
while np.linalg.norm(Rnew - R) > 1e-14 and iter < maxiter:
        iter = iter + 1
        R = Rnew
        Z = np.matmul(V , R.T)

        # Threshold to solve sparse approximation problem
        for i in range(r):
                Si = soft_threshold(Z[:,i], mu)
                S[:, i] = Si / np.linalg.norm(Si)
        # Polar decomposition to solve Procrustes problem
        U, _, Vt = np.linalg.svd(np.matmul(S.T , V), full_matrices=False)
        Rnew = np.matmul(U , Vt)

# Choose correct parity of vectors and scale so largest value is 1
for i in range(r):
        S[:, i] = S[:, i] * np.sign(sum(S[:, i]))
        S[:, i] = S[:, i] / np.max(S[:, i])

# Sort so that most reliable vectors appear first
ind = np.argsort(np.min(S, axis=0))
S = S[:, ind]

    

In [177]:
S

array([[-0.        ,  0.        ,  0.        ],
       [-0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.28251653,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        , -0.        ,  0.82452219],
       [-0.05688563,  0.        ,  0.23001618],
       [ 0.35860632,  0.        ,  0.        ],
       [ 0.        , -0.        ,  0.        ],
       [-0.        , -0.        ,  0.97639582],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.37562904],
       [ 0.        ,  0.04235165,  0.        ],
       [ 0.        , -0.        ,  0.        ],
       [-0.        ,  0.93917397,  0.        ],
       [ 0.        , -0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.41816185],
       [ 0.07006412,  0.35656896, -0.        ],
       [-0.35452751,  0.40612526,  0.        ],
       [ 0.        ,  0.71113735, -0.        ],
       [ 0.06259606,  0.12203169, -0.        ],
       [-0.        ,  0.        ,  0.217

In [178]:
S2, R2 = methods.SEBA(V_orig)

In [179]:
S2

array([[-0.        ,  0.        ,  0.        ],
       [-0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.28251653,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        , -0.        ,  0.82452219],
       [-0.05688563,  0.        ,  0.23001618],
       [ 0.35860632,  0.        ,  0.        ],
       [ 0.        , -0.        ,  0.        ],
       [-0.        , -0.        ,  0.97639582],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.37562904],
       [ 0.        ,  0.04235165,  0.        ],
       [ 0.        , -0.        ,  0.        ],
       [-0.        ,  0.93917397,  0.        ],
       [ 0.        , -0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.41816185],
       [ 0.07006412,  0.35656896, -0.        ],
       [-0.35452751,  0.40612526,  0.        ],
       [ 0.        ,  0.71113735, -0.        ],
       [ 0.06259606,  0.12203169, -0.        ],
       [-0.        ,  0.        ,  0.217

In [180]:
print(np.min(S, axis=0))
np.argsort(np.min(S, axis=0))

[-0.35452751 -0.25502339 -0.0695569 ]


array([0, 1, 2])

In [181]:
# np.dot(V[:,0],V[:,1]) # check orthonormality
# np.linalg.norm(V)
# np.matmul(V,np.eye(r))
soft_threshold(V[:,0], mu)
# np.max([np.abs(V[1,0])-mu,0])
# len(np.shape(0))

# soft_threshold(-0.6,0.5)

array([-0.        , -0.        , -0.00543554, -0.0034322 , -0.04238137,
       -0.        , -0.01750587, -0.        , -0.04682486, -0.02386785,
       -0.03726026, -0.        , -0.        , -0.        , -0.00965437,
       -0.02739554, -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.0044613 , -0.        , -0.06589141, -0.        ,
       -0.01904367, -0.        , -0.00821744, -0.        , -0.04200819,
       -0.05473125, -0.        , -0.05065874, -0.07380749, -0.        ,
       -0.0683592 , -0.        , -0.        , -0.        , -0.        ,
       -0.01247904, -0.        , -0.03229128, -0.        , -0.03808252,
       -0.01598029, -0.02892586, -0.        , -0.01969824, -0.05288829,
       -0.        , -0.00630912, -0.00189407, -0.        , -0.        ,
       -0.        , -0.        , -0.01759067, -0.01549297, -0.0726666 ,
       -0.        , -0.00560507, -0.        , -0.0375265 , -0.01160488,
       -0.04212863, -0.        , -0.        , -0.        , -0.  